# Part1: Collecting Data from Skeleton Info

In [56]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from highcharts import Highchart

In [57]:
def getJsonList(filePath):
    fileList = []
    for i,j,k in os.walk(filePath):
        fileList = k
    return fileList

def calcWeight(item):
    zeros = 0
    for i in item['pose_keypoints_2d']:
        if i == 0:
            zeros += 1
    for i in item['face_keypoints_2d']:
        if i == 0:
            zeros +=1 
    return zeros

def getJsonForOneFrame(fileName, filePath):
    l = getJsonList(filePath)
    with open(fileName,'r') as f:
        temp = json.loads(f.read())
        allPeople = temp['people']
    
    if len(allPeople) == 0:
        empty = []
        for i in range(190):
            empty.append(0)
        return empty
    target = allPeople[0]
    for dic in allPeople:
        if calcWeight(dic) < calcWeight(target):
            target = dic
    matrix = []
    if(len(target['pose_keypoints_2d']) == 0):
        for i in range(50):
            matrix.append(0)
    else:
        for i in range(75):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['pose_keypoints_2d'][i])
                
    if(len(target['face_keypoints_2d']) == 0):
        for i in range(140):
            matrix.append(0)
    else:
        for i in range(210):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['face_keypoints_2d'][i])
    return matrix

In [58]:
def getLable(dataCat, name, init):
    #print(name, init)
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        if row['name'] == name:
            mark = row['mark']
            mark = mark.split('-')

            if len(mark) % 2 != 0 or len(mark) == 0:
                print("ERROR")
                return None
            flag = 0
            for i in range(30):
                slow = 0
                fast = 1
                while(fast < len(mark)):
                    small = mark[slow]
                    big = mark[fast]
                    if int(small) <= int(init) + i <= int(big):
                        flag += 1
                        break
                    else:
                        slow += 2
                        fast += 2
            #print(flag)
    if flag >= 15:
        return True
    else:
        return False

def writeData(path, pathType,fullDfList):
    fileList = getJsonList(path)
    fileList = set(fileList)
    print(len(fileList))
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        #temp = int(row['name'][:-1])
        if(row["type"] != pathType):
            pass
        else:
            startName = row["name"]+"_"+'0'.zfill(12)+"_keypoints.json"
            endName = row["name"]+"_"+'30'.zfill(12)+"_keypoints.json"
            if endName and endName not in fileList:
                print('Video ',row["name"]," is not long enough")
            else:
                initFrame = 0
                while(row["name"]+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
                    matrix = []
                    for i in range(30):
                        if (i+1)% 3 == 0:
                            file = row["name"]+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
                            matrixLine = getJsonForOneFrame(path+'/'+file,path)
                            matrix.append(matrixLine)
                           
                        else:
                            pass
                    if row['type'] == 'shortyes':
                        label = True
                    elif row['type'] == 'shortno':
                        label = False
                    else:
                        label = getLable(dataCat,row['name'], initFrame)
                    data = [[label, matrix]]
                    print("Data:",label, row['name'], initFrame,'matrix:',len(matrix),len(matrix[0]))
                    df = pd.DataFrame(data,columns=['lable','matrix'])
                    fullDfList.append(df)
                    initFrame += 30
    return fullDfList

In [4]:
fullDfList = []
fullDfList = writeData('dataset/confusing', 'shortno',fullDfList)
df = pd.concat(fullDfList, axis=0 )
df.reset_index(drop=True, inplace=True)
print(df)
df.to_json("dataset/confusing.json")


fullDfList = []
fullDfList = writeData('dataset/noface', 'shortno',fullDfList)
df = pd.concat(fullDfList, axis=0 )
df.reset_index(drop=True, inplace=True)
print(df)
df.to_json("dataset/noface.json")


fullDfList = []
fullDfList = writeData('dataset/realsilent', 'shortno',fullDfList)
df = pd.concat(fullDfList, axis=0 )
df.reset_index(drop=True, inplace=True)
print(df)
df.to_json("dataset/realsilent.json")


fullDfList = []
fullDfList = writeData('dataset/realtalking', 'shortyes',fullDfList)
df = pd.concat(fullDfList, axis=0 )
df.reset_index(drop=True, inplace=True)
print(df)
df.to_json("dataset/realtalking.json")

65689
Data: False confusing 0 matrix: 10 190
Data: False confusing 30 matrix: 10 190
Data: False confusing 60 matrix: 10 190
Data: False confusing 90 matrix: 10 190
Data: False confusing 120 matrix: 10 190
Data: False confusing 150 matrix: 10 190
Data: False confusing 180 matrix: 10 190
Data: False confusing 210 matrix: 10 190
Data: False confusing 240 matrix: 10 190
Data: False confusing 270 matrix: 10 190
Data: False confusing 300 matrix: 10 190
Data: False confusing 330 matrix: 10 190
Data: False confusing 360 matrix: 10 190
Data: False confusing 390 matrix: 10 190
Data: False confusing 420 matrix: 10 190
Data: False confusing 450 matrix: 10 190
Data: False confusing 480 matrix: 10 190
Data: False confusing 510 matrix: 10 190
Data: False confusing 540 matrix: 10 190
Data: False confusing 570 matrix: 10 190
Data: False confusing 600 matrix: 10 190
Data: False confusing 630 matrix: 10 190
Data: False confusing 660 matrix: 10 190
Data: False confusing 690 matrix: 10 190
Data: False con

Data: False confusing 5880 matrix: 10 190
Data: False confusing 5910 matrix: 10 190
Data: False confusing 5940 matrix: 10 190
Data: False confusing 5970 matrix: 10 190
Data: False confusing 6000 matrix: 10 190
Data: False confusing 6030 matrix: 10 190
Data: False confusing 6060 matrix: 10 190
Data: False confusing 6090 matrix: 10 190
Data: False confusing 6120 matrix: 10 190
Data: False confusing 6150 matrix: 10 190
Data: False confusing 6180 matrix: 10 190
Data: False confusing 6210 matrix: 10 190
Data: False confusing 6240 matrix: 10 190
Data: False confusing 6270 matrix: 10 190
Data: False confusing 6300 matrix: 10 190
Data: False confusing 6330 matrix: 10 190
Data: False confusing 6360 matrix: 10 190
Data: False confusing 6390 matrix: 10 190
Data: False confusing 6420 matrix: 10 190
Data: False confusing 6450 matrix: 10 190
Data: False confusing 6480 matrix: 10 190
Data: False confusing 6510 matrix: 10 190
Data: False confusing 6540 matrix: 10 190
Data: False confusing 6570 matrix:

Data: False confusing 11700 matrix: 10 190
Data: False confusing 11730 matrix: 10 190
Data: False confusing 11760 matrix: 10 190
Data: False confusing 11790 matrix: 10 190
Data: False confusing 11820 matrix: 10 190
Data: False confusing 11850 matrix: 10 190
Data: False confusing 11880 matrix: 10 190
Data: False confusing 11910 matrix: 10 190
Data: False confusing 11940 matrix: 10 190
Data: False confusing 11970 matrix: 10 190
Data: False confusing 12000 matrix: 10 190
Data: False confusing 12030 matrix: 10 190
Data: False confusing 12060 matrix: 10 190
Data: False confusing 12090 matrix: 10 190
Data: False confusing 12120 matrix: 10 190
Data: False confusing 12150 matrix: 10 190
Data: False confusing 12180 matrix: 10 190
Data: False confusing 12210 matrix: 10 190
Data: False confusing 12240 matrix: 10 190
Data: False confusing 12270 matrix: 10 190
Data: False confusing 12300 matrix: 10 190
Data: False confusing 12330 matrix: 10 190
Data: False confusing 12360 matrix: 10 190
Data: False

Data: False confusing 17430 matrix: 10 190
Data: False confusing 17460 matrix: 10 190
Data: False confusing 17490 matrix: 10 190
Data: False confusing 17520 matrix: 10 190
Data: False confusing 17550 matrix: 10 190
Data: False confusing 17580 matrix: 10 190
Data: False confusing 17610 matrix: 10 190
Data: False confusing 17640 matrix: 10 190
Data: False confusing 17670 matrix: 10 190
Data: False confusing 17700 matrix: 10 190
Data: False confusing 17730 matrix: 10 190
Data: False confusing 17760 matrix: 10 190
Data: False confusing 17790 matrix: 10 190
Data: False confusing 17820 matrix: 10 190
Data: False confusing 17850 matrix: 10 190
Data: False confusing 17880 matrix: 10 190
Data: False confusing 17910 matrix: 10 190
Data: False confusing 17940 matrix: 10 190
Data: False confusing 17970 matrix: 10 190
Data: False confusing 18000 matrix: 10 190
Data: False confusing 18030 matrix: 10 190
Data: False confusing 18060 matrix: 10 190
Data: False confusing 18090 matrix: 10 190
Data: False

Data: False confusing 23160 matrix: 10 190
Data: False confusing 23190 matrix: 10 190
Data: False confusing 23220 matrix: 10 190
Data: False confusing 23250 matrix: 10 190
Data: False confusing 23280 matrix: 10 190
Data: False confusing 23310 matrix: 10 190
Data: False confusing 23340 matrix: 10 190
Data: False confusing 23370 matrix: 10 190
Data: False confusing 23400 matrix: 10 190
Data: False confusing 23430 matrix: 10 190
Data: False confusing 23460 matrix: 10 190
Data: False confusing 23490 matrix: 10 190
Data: False confusing 23520 matrix: 10 190
Data: False confusing 23550 matrix: 10 190
Data: False confusing 23580 matrix: 10 190
Data: False confusing 23610 matrix: 10 190
Data: False confusing 23640 matrix: 10 190
Data: False confusing 23670 matrix: 10 190
Data: False confusing 23700 matrix: 10 190
Data: False confusing 23730 matrix: 10 190
Data: False confusing 23760 matrix: 10 190
Data: False confusing 23790 matrix: 10 190
Data: False confusing 23820 matrix: 10 190
Data: False

...

Data: False confusing 40350 matrix: 10 190
Data: False confusing 40380 matrix: 10 190
Data: False confusing 40410 matrix: 10 190
Data: False confusing 40440 matrix: 10 190
Data: False confusing 40470 matrix: 10 190
Data: False confusing 40500 matrix: 10 190
Data: False confusing 40530 matrix: 10 190
Data: False confusing 40560 matrix: 10 190
Data: False confusing 40590 matrix: 10 190
Data: False confusing 40620 matrix: 10 190
Data: False confusing 40650 matrix: 10 190
Data: False confusing 40680 matrix: 10 190
Data: False confusing 40710 matrix: 10 190
Data: False confusing 40740 matrix: 10 190
Data: False confusing 40770 matrix: 10 190
Data: False confusing 40800 matrix: 10 190
Data: False confusing 40830 matrix: 10 190
Data: False confusing 40860 matrix: 10 190
Data: False confusing 40890 matrix: 10 190
Data: False confusing 40920 matrix: 10 190
Data: False confusing 40950 matrix: 10 190
Data: False confusing 40980 matrix: 10 190
Data: False confusing 41010 matrix: 10 190
Data: False

Data: False confusing 46080 matrix: 10 190
Data: False confusing 46110 matrix: 10 190
Data: False confusing 46140 matrix: 10 190
Data: False confusing 46170 matrix: 10 190
Data: False confusing 46200 matrix: 10 190
Data: False confusing 46230 matrix: 10 190
Data: False confusing 46260 matrix: 10 190
Data: False confusing 46290 matrix: 10 190
Data: False confusing 46320 matrix: 10 190
Data: False confusing 46350 matrix: 10 190
Data: False confusing 46380 matrix: 10 190
Data: False confusing 46410 matrix: 10 190
Data: False confusing 46440 matrix: 10 190
Data: False confusing 46470 matrix: 10 190
Data: False confusing 46500 matrix: 10 190
Data: False confusing 46530 matrix: 10 190
Data: False confusing 46560 matrix: 10 190
Data: False confusing 46590 matrix: 10 190
Data: False confusing 46620 matrix: 10 190
Data: False confusing 46650 matrix: 10 190
Data: False confusing 46680 matrix: 10 190
Data: False confusing 46710 matrix: 10 190
Data: False confusing 46740 matrix: 10 190
Data: False

Data: False confusing 51810 matrix: 10 190
Data: False confusing 51840 matrix: 10 190
Data: False confusing 51870 matrix: 10 190
Data: False confusing 51900 matrix: 10 190
Data: False confusing 51930 matrix: 10 190
Data: False confusing 51960 matrix: 10 190
Data: False confusing 51990 matrix: 10 190
Data: False confusing 52020 matrix: 10 190
Data: False confusing 52050 matrix: 10 190
Data: False confusing 52080 matrix: 10 190
Data: False confusing 52110 matrix: 10 190
Data: False confusing 52140 matrix: 10 190
Data: False confusing 52170 matrix: 10 190
Data: False confusing 52200 matrix: 10 190
Data: False confusing 52230 matrix: 10 190
Data: False confusing 52260 matrix: 10 190
Data: False confusing 52290 matrix: 10 190
Data: False confusing 52320 matrix: 10 190
Data: False confusing 52350 matrix: 10 190
Data: False confusing 52380 matrix: 10 190
Data: False confusing 52410 matrix: 10 190
Data: False confusing 52440 matrix: 10 190
Data: False confusing 52470 matrix: 10 190
Data: False

Data: False confusing 57540 matrix: 10 190
Data: False confusing 57570 matrix: 10 190
Data: False confusing 57600 matrix: 10 190
Data: False confusing 57630 matrix: 10 190
Data: False confusing 57660 matrix: 10 190
Data: False confusing 57690 matrix: 10 190
Data: False confusing 57720 matrix: 10 190
Data: False confusing 57750 matrix: 10 190
Data: False confusing 57780 matrix: 10 190
Data: False confusing 57810 matrix: 10 190
Data: False confusing 57840 matrix: 10 190
Data: False confusing 57870 matrix: 10 190
Data: False confusing 57900 matrix: 10 190
Data: False confusing 57930 matrix: 10 190
Data: False confusing 57960 matrix: 10 190
Data: False confusing 57990 matrix: 10 190
Data: False confusing 58020 matrix: 10 190
Data: False confusing 58050 matrix: 10 190
Data: False confusing 58080 matrix: 10 190
Data: False confusing 58110 matrix: 10 190
Data: False confusing 58140 matrix: 10 190
Data: False confusing 58170 matrix: 10 190
Data: False confusing 58200 matrix: 10 190
Data: False

Data: False confusing 63270 matrix: 10 190
Data: False confusing 63300 matrix: 10 190
Data: False confusing 63330 matrix: 10 190
Data: False confusing 63360 matrix: 10 190
Data: False confusing 63390 matrix: 10 190
Data: False confusing 63420 matrix: 10 190
Data: False confusing 63450 matrix: 10 190
Data: False confusing 63480 matrix: 10 190
Data: False confusing 63510 matrix: 10 190
Data: False confusing 63540 matrix: 10 190
Data: False confusing 63570 matrix: 10 190
Data: False confusing 63600 matrix: 10 190
Data: False confusing 63630 matrix: 10 190
Data: False confusing 63660 matrix: 10 190
Data: False confusing 63690 matrix: 10 190
Data: False confusing 63720 matrix: 10 190
Data: False confusing 63750 matrix: 10 190
Data: False confusing 63780 matrix: 10 190
Data: False confusing 63810 matrix: 10 190
Data: False confusing 63840 matrix: 10 190
Data: False confusing 63870 matrix: 10 190
Data: False confusing 63900 matrix: 10 190
Data: False confusing 63930 matrix: 10 190
Data: False

In [19]:
def getTrainTest(a,b,c,d,e):    
    fulldf = pd.read_json(a)
    print(fulldf)
    #cutdf = fulldf.sample(frac=0.1, replace=False)
    cutdf = fulldf
    print(cutdf)
    x = cutdf['matrix']
    y = cutdf['lable']
    x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.3)

    x_train.reset_index(drop=True, inplace=True)
    x_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    print('1')

    def cleanX(raw):
        length = len(raw)
        temp = np.zeros((length, 10,190))
        for i in range(len(raw)):
            for j in range(10):
                for k in range(190):
                    temp[i][j][k] = raw[i][j][k]
            print("progress:{0}%".format(round((i + 1) * 100 / len(raw))), end="\r")
        return temp
    x_test = cleanX(x_test)
    x_train = cleanX(x_train)
    #print(temp)
    def cleanY(raw):
        res = []
        for i in range(len(raw)):
            if raw[i] == True:
                res.append(1)
            else:
                res.append(0)
        res = np.array(res)
        return res
    y_train = cleanY(y_train)
    y_test = cleanY(y_test)
    print(x_train.shape)
    print(x_test.shape)
    print(y_train.shape)
    print(y_test.shape)


    np.save(b,x_train)
    np.save(c,x_test)
    np.save(d,y_train)
    np.save(e,y_test)

In [20]:
getTrainTest("dataset/confusing.json","dataset/x_train_confusing.npy","dataset/x_test_confusing.npy","dataset/y_train_confusing.npy","dataset/y_test_confusing.npy")
getTrainTest("dataset/noface.json","dataset/x_train_noface.npy","dataset/x_test_noface.npy","dataset/y_train_noface.npy","dataset/y_test_noface.npy")
getTrainTest("dataset/realsilent.json","dataset/x_train_realsilent.npy","dataset/x_test_realsilent.npy","dataset/y_train_realsilent.npy","dataset/y_test_realsilent.npy")
getTrainTest("dataset/realtalking.json","dataset/x_train_realtalking.npy","dataset/x_test_realtalking.npy","dataset/y_train_realtalking.npy","dataset/y_test_realtalking.npy")

      lable                                             matrix
0     False  [[652.888, 294.368, 664.598, 478.527, 478.547,...
1     False  [[723.447, 321.731, 684.268, 492.202, 504.003,...
2     False  [[707.729, 319.824, 676.361, 480.483, 492.216,...
3     False  [[697.889, 327.643, 676.372, 482.48, 490.267, ...
4     False  [[699.906, 331.551, 680.304, 480.537, 496.166,...
...     ...                                                ...
2184  False  [[478.491, 478.494, 449.118, 715.604, 25.9858,...
2185  False  [[462.846, 478.561, 0, 0, 49.474, 715.598, 0, ...
2186  False  [[476.583, 476.617, 447.162, 715.604, 0, 0, 0,...
2187  False  [[496.159, 476.545, 449.159, 715.597, 0, 0, 0,...
2188  False  [[482.463, 488.346, 0, 0, 0, 0, 0, 0, 0, 0, 0,...

[2189 rows x 2 columns]
      lable                                             matrix
0     False  [[652.888, 294.368, 664.598, 478.527, 478.547,...
1     False  [[723.447, 321.731, 684.268, 492.202, 504.003,...
2     False  [[707.729, 319.82

In [59]:
xtr1 = np.load("dataset/x_train_confusing.npy")
xtr2 = np.load("dataset/x_train_noface.npy")
xtr3 = np.load("dataset/x_train_realsilent.npy")
xtr4 = np.load("dataset/x_train_realtalking.npy")

xte1 = np.load("dataset/x_test_confusing.npy")
xte2 = np.load("dataset/x_test_noface.npy")
xte3 = np.load("dataset/x_test_realsilent.npy")
xte4 = np.load("dataset/x_test_realtalking.npy")

ytr1 = np.load("dataset/y_train_confusing.npy")
ytr2 = np.load("dataset/y_train_noface.npy")
ytr3 = np.load("dataset/y_train_realsilent.npy")
ytr4 = np.load("dataset/y_train_realtalking.npy")

yte1 = np.load("dataset/y_test_confusing.npy")#0
yte2 = np.load("dataset/y_test_noface.npy")#0
yte3 = np.load("dataset/y_test_realsilent.npy")#0
yte4 = np.load("dataset/y_test_realtalking.npy")#0

yte1t = np.load("dataset/y_test_confusing_true.npy")#1
yte2t = np.load("dataset/y_test_noface_true.npy")#1
yte3t = np.load("dataset/y_test_realsilent_true.npy")#1
yte4t = np.load("dataset/y_test_realtalking_true.npy")#1

ytr1t = np.load("dataset/y_train_confusing_true.npy")#1
ytr2t = np.load("dataset/y_train_noface_true.npy")#1
ytr3t = np.load("dataset/y_train_realsilent_true.npy")#1
ytr4t = np.load("dataset/y_train_realtalking_true.npy")#1



In [60]:
def ProcessSilentData(x_train, x_test):
    def proc(x_train_silent):
        length = len(x_train_silent)
        temp = np.zeros((length, 10,46))
        for i in range(len(x_train_silent)):
            for j in range(10):
                for k in range(190):
                    if 146 <= k <= 185:
                        temp[i][j][k-146] = x_train_silent[i][j][k]
                temp[i][j][40] = x_train_silent[i][j][67]#face
                temp[i][j][41] = x_train_silent[i][j][68]
                temp[i][j][42] = x_train_silent[i][j][8]#left hand
                temp[i][j][43] = x_train_silent[i][j][9]
                temp[i][j][44] = x_train_silent[i][j][14]#right hand
                temp[i][j][45] = x_train_silent[i][j][15]
                
        return temp
    x_train_40 = proc(x_train)
    x_test_40 = proc(x_test)
    x_train = x_train_40
    x_test = x_test_40

    #define distance between two points
    def calDis(material, a, b):
        #material = x_train[0][0]
        ax = (a-1) *2
        ay = ax + 1
        bx = (b-1) *2
        by = bx + 1
        d1 = abs(material[ax] - material[bx])
        d2 = abs(material[ay] - material[by])
        dis = np.sqrt(d1*d1 + d2*d2)
        return dis

    def getSilent(x):#x_train[0][0]
        mouthLen = calDis(x, 13, 17)
        sideLen = calDis(x, 1, 12) + calDis(x, 1, 2) + calDis(x, 8, 7) + calDis(x, 6, 7)
        mouthWid = calDis(x, 14, 20) + calDis(x, 15, 19) + calDis(x, 16, 18)
        handLen = calDis(x, 21, 22) + calDis(x, 21, 23)
        if mouthLen == 0:
            silentWeight = 0
        else:
            silentWeight = mouthWid / mouthLen
        if sideLen == 0:
            sideWeight = 0
            handWeight = 0
        else:
            sideWeight = mouthWid / sideLen
            handWeight = handLen / sideLen
        if sideLen == 0:
            return [-1, -1]
        else:
            return [sideWeight,silentWeight]

    def proX(x_train):
        pro_x_train = np.zeros((len(x_train), 10, 4))
        for i in range(len(x_train)):
            for j in range(len(x_train[i])):
                var0, var1 = getSilent(x_train[i][j])
                pro_x_train[i][j][0] = var0
                pro_x_train[i][j][1] = var1
        for i in range(len(pro_x_train)):#10 2
            varList1 = []
            varList2 = []
            for j in range(10):
                varList1.append(pro_x_train[i][j][0])
                varList2.append(pro_x_train[i][j][1])
            var2 = np.var(varList1)
            var3 = np.var(varList2)
            for j in range(10):
                pro_x_train[i][j][2] = var2
                pro_x_train[i][j][3] = var3
        return pro_x_train
    return proX(x_train), proX(x_test)

In [67]:
x_train_talking = np.concatenate((xtr2, xtr3, xtr4), axis=0)
x_test_talking = np.concatenate((xtr2, xte3, xte4), axis=0)
y_train_talking = np.concatenate((ytr2, ytr3, ytr4t), axis=0)
y_test_talking = np.concatenate((ytr2, yte3, yte4t), axis=0)


x_train_talking,x_test_talking = ProcessSilentData(x_train_talking, x_test_talking)###reduced

np.save("dataset/data/x_train_talking.npy",x_train_talking)
np.save("dataset/data/x_test_talking.npy",x_test_talking)
np.save("dataset/data/y_train_talking.npy",y_train_talking)
np.save("dataset/data/y_test_talking.npy",y_test_talking)

In [68]:
x_train_confusing = np.concatenate((xtr1, xtr4), axis=0)
x_test_confusing = np.concatenate((xte1, xte4), axis=0)
y_train_confusing = np.concatenate((ytr1t, ytr4), axis=0)
y_test_confusing = np.concatenate((yte1t, yte4), axis=0)


x_train_confusing,x_test_confusing = ProcessSilentData(x_train_confusing, x_test_confusing)###reduced

np.save("dataset/data/x_train_confusing.npy",x_train_confusing)
np.save("dataset/data/x_test_confusing.npy",x_test_confusing)
np.save("dataset/data/y_train_confusing.npy",y_train_confusing)
np.save("dataset/data/y_test_confusing.npy",y_test_confusing)

In [72]:
x_train_ensemble = np.concatenate((xtr1, xtr2, xtr3, xtr4), axis=0)
x_test_ensemble = np.concatenate((xte1, xte2, xte3, xte4), axis=0)
y_train_ensemble = np.concatenate((ytr1, ytr2, ytr3, ytr4t), axis=0)
y_test_ensemble = np.concatenate((yte1, yte2, yte3, yte4t), axis=0)


x_train_ensemble,x_test_ensemble = ProcessSilentData(x_train_ensemble, x_test_ensemble)###reduced

np.save("dataset/data/x_test_ensemble.npy",x_train_ensemble)
np.save("dataset/data/x_train_ensemble.npy",x_test_ensemble)
np.save("dataset/data/y_test_ensemble.npy",y_train_ensemble)
np.save("dataset/data/y_train_ensemble.npy",y_test_ensemble)

print(y_train_ensemble)

[0 0 0 ... 1 1 1]
